In [5]:
import torch
import torch.nn as nn
from torch_utils import View

In [ ]:
img_1 = torch.ones(7, 1, 64, 64)
img_2 = torch.zeros(7, 1, 64, 64)


In [28]:
trunk = [
            nn.Conv2d(1, 128, kernel_size=4, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 10, kernel_size=4, stride=1),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            View(280),
]
head = [
            nn.Linear(280 * 2, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
    ]

In [29]:
_ = img_1
for l in trunk:
    _ = l(_)
print(_.shape)
_ = torch.cat([_, _], dim=-1)
print(_.shape)
for l in head:
    _ = l(_)
print(_.shape)


torch.Size([1, 280])
torch.Size([1, 560])
torch.Size([1, 1])


In [36]:

class GlobalMetricConvDeepKernel(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()

        self.trunk = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=4, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 10, kernel_size=4, stride=1),
            nn.BatchNorm2d(10),
            nn.ReLU(),
            View(40)
        )
        self.head = nn.Sequential(
            nn.Linear(40 * 2, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )

    def encode(self, x):
        return self.trunk(x)

    def forward(self, x, x_prime):
        x, x_prime = torch.broadcast_tensors(x, x_prime)
        *b, C, H, W = x.shape
        _ = torch.cat([self.trunk(x), self.trunk(x_prime)], dim=-1)
        print(b)
        return self.head(_).reshape(*b, 1)

In [37]:
m = GlobalMetricConvDeepKernel(1, 32)


In [38]:
_ = m(img_1, img_2)
print(_.shape)


[7]
torch.Size([7, 1])
